# 사용자 맞춤 뉴스 추천 모델링

In [116]:
import pandas as pd
import numpy as np

In [110]:
df_bosa = pd.read_csv("./bosa_news_all.csv")
df_bosa.head()

,news_title,news_url,news_when,news_topic,news_paper,news_contents
0,"한국쿄와기린, 세계 희귀질환의 날 행사 참여",http://www.bosa.co.kr/news/articleView.html?id...,2024.02.29,의료/법안,의학신문,[의학신문·일간보사=김영주 기자]‘세계 희귀질환의 날’을 맞아 한국쿄와기린은 그룹에...
1,"광동제약, ‘세계 희귀질환의 날’ 기념 환아 작품 전시회 개최",http://www.bosa.co.kr/news/articleView.html?id...,2024.02.29,신약/개발,의학신문,[의학신문·일간보사=김영주 기자]광동제약(대표 최성원)은 ‘세계 희귀질환의 날’을 ...
2,"순천향대 부천병원 신영림 교수, ‘질병관리청장 표창’ 수상",http://www.bosa.co.kr/news/articleView.html?id...,2024.02.29,심포지엄/행사,의학신문,[의학신문·일간보사=이상만 기자]순천향대 부천병원 소아청소년과 신영림 교수(사진)가...
3,"창립 70주년 한독, ‘THANKS CAMPAIGN’ 실시",http://www.bosa.co.kr/news/articleView.html?id...,2024.02.29,의료/법안,의학신문,"[의학신문·일간보사=김정일 기자] 한독(대표이사 김영진, 백진기)이 창립 70주년을..."
4,[세계 희귀질환의 날] 국가‧사회‧이웃‧의료진‧가족 함께 가야 한다,http://www.bosa.co.kr/news/articleView.html?id...,2024.02.29,신약/개발,의학신문,[의학신문·일간보사=오인규 기자] “이건 신이 실수를 하신 거다. 그러니 너무 자책...


In [112]:
df_bosa['news_when'] = pd.to_datetime(df_bosa['news_when'])

In [113]:
df_bosa.head()

,news_title,news_url,news_when,news_topic,news_paper,news_contents
0,"한국쿄와기린, 세계 희귀질환의 날 행사 참여",http://www.bosa.co.kr/news/articleView.html?id...,2024-02-29,의료/법안,의학신문,[의학신문·일간보사=김영주 기자]‘세계 희귀질환의 날’을 맞아 한국쿄와기린은 그룹에...
1,"광동제약, ‘세계 희귀질환의 날’ 기념 환아 작품 전시회 개최",http://www.bosa.co.kr/news/articleView.html?id...,2024-02-29,신약/개발,의학신문,[의학신문·일간보사=김영주 기자]광동제약(대표 최성원)은 ‘세계 희귀질환의 날’을 ...
2,"순천향대 부천병원 신영림 교수, ‘질병관리청장 표창’ 수상",http://www.bosa.co.kr/news/articleView.html?id...,2024-02-29,심포지엄/행사,의학신문,[의학신문·일간보사=이상만 기자]순천향대 부천병원 소아청소년과 신영림 교수(사진)가...
3,"창립 70주년 한독, ‘THANKS CAMPAIGN’ 실시",http://www.bosa.co.kr/news/articleView.html?id...,2024-02-29,의료/법안,의학신문,"[의학신문·일간보사=김정일 기자] 한독(대표이사 김영진, 백진기)이 창립 70주년을..."
4,[세계 희귀질환의 날] 국가‧사회‧이웃‧의료진‧가족 함께 가야 한다,http://www.bosa.co.kr/news/articleView.html?id...,2024-02-29,신약/개발,의학신문,[의학신문·일간보사=오인규 기자] “이건 신이 실수를 하신 거다. 그러니 너무 자책...


In [114]:
df_bosa.dropna(subset=['news_title'], inplace=True)

pandas._libs.tslibs.timestamps.Timestamp

In [ ]:
df_bosa.to_csv('bosa_news_all.csv', index=False)

In [33]:
df_bosa['news_topic'].value_counts()  #제약/개발 #병원 #법령/정책 #의료기기 #학회/학술 #

news_topic
의료/법안      874
신약/개발      824
심포지엄/행사    727
Name: count, dtype: int64

In [34]:
df_bosa.info

<bound method DataFrame.info of                                  news_title  \
0                  한국쿄와기린, 세계 희귀질환의 날 행사 참여   
1        광동제약, ‘세계 희귀질환의 날’ 기념 환아 작품 전시회 개최   
2          순천향대 부천병원 신영림 교수, ‘질병관리청장 표창’ 수상   
3          창립 70주년 한독, ‘THANKS CAMPAIGN’ 실시   
4     [세계 희귀질환의 날] 국가‧사회‧이웃‧의료진‧가족 함께 가야 한다   
...                                     ...   
2420                     26일 `희귀질환 치료' 심포지엄   
2421                  `희귀^난치성질환자 연합회' 구성 추진   
2422                      책임운영기관 평가 “적정성 결여   
2423                    먹지도 굶지도 못하는 희귀질환 발견   
2424                희귀의약품센터 공익성 기부금 대상단체 지정   

                                               news_url   news_when  \
0     http://www.bosa.co.kr/news/articleView.html?id...  2024.02.29   
1     http://www.bosa.co.kr/news/articleView.html?id...  2024.02.29   
2     http://www.bosa.co.kr/news/articleView.html?id...  2024.02.29   
3     http://www.bosa.co.kr/news/articleView.html?id...  2024.02.29   
4     http://www.bosa.co.kr/news/articleView.htm

In [35]:
df_bosa.isnull().sum()

news_title         0
news_url           0
news_when          0
news_topic         0
news_paper         0
news_contents    283
dtype: int64

In [36]:
df_bosa = df_bosa.reset_index().drop(columns='index')

In [37]:
df_bosa_test1 = df_bosa.copy()

In [38]:
df_bosa_test1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2425 entries, 0 to 2424
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   news_title     2425 non-null   object
 1   news_url       2425 non-null   object
 2   news_when      2425 non-null   object
 3   news_topic     2425 non-null   object
 4   news_paper     2425 non-null   object
 5   news_contents  2142 non-null   object
dtypes: object(6)
memory usage: 113.8+ KB


## 1. 군집화 후 토픽 분류
- 우선 1차적으로 군집을 만들어 어떠한 topic으로 분류되는지 확인해보자

In [39]:
df_bosa_test1 = df_bosa.dropna(subset=['news_title'])

### 토큰화

In [40]:
# 불용어 리스트 생성 '녹십자'
stopwords = ['서울대', '희귀질환', '희귀', '대다',  '케다', '소아', '생명', '한국', '한미','사노피', '하다', '급여', '국내', '샤이어',  '스케', '세포'
            , '병원',  '질환',  '한독', '화이자제약',  '전달', '질병', '인하대병원',  '관리', '다국적', '환자', '지정', '치료'
            , '오다', '헌터', '작년', '브리', '위해', '베다', '받다', '심평원', '코로나', '건보', '화순', '전남대', '실시', '자임','녹십자'
            ] #추가 생성 필요
f=open('./korean_stopwords_basic.txt') #기본적으로 제공되는 한국어 불용어 리스트 파일
lines = f.readlines()
for line in lines:
    line = line.strip()
    stopwords.append(line)
f.close()

In [41]:
from konlpy.tag import Okt
okt = Okt()
#토크나이징 함수 정의
def tokenizer(raw, pos=['Noun', 'Verb'], stopword=stopwords):
    return [
        word for word, tag in okt.pos(raw, norm=True, stem=True)
        if len(word) >1 and tag in pos and word not in stopword
    ]

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfvectorizer = TfidfVectorizer(tokenizer=tokenizer, max_df=0.95, min_df=3)

In [43]:
len(df_bosa_test1['news_title'])

2425

In [44]:
features = tfidfvectorizer.fit_transform(df_bosa_test1['news_title'])
# news_title에 있는 내용을 백터&토큰화

/usr/local/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [45]:
features.shape

(2425, 997)

In [46]:
features

<2425x997 sparse matrix of type '<class 'numpy.float64'>'
	with 9408 stored elements in Compressed Sparse Row format>

### perplexity를 이용해 확인하기

In [47]:
import pyLDAvis
import pyLDAvis.lda_model
from sklearn.decomposition import LatentDirichletAllocation

In [48]:
from sklearn.feature_extraction.text import CountVectorizer

In [49]:
def perplexity(corpus):
    best_ppx = float('inf')
    best_topic_num = None
    for num_topics in range(4,21):
        count_vectorizer = CountVectorizer(max_features=100) # 적절한 max_feature 선택
        X_count = count_vectorizer.fit_transform(corpus)
        # LDA 모델 초기화
        lda_model = LatentDirichletAllocation(n_components = num_topics, random_state=55)
        lda_model.fit(X_count)
        ppx = lda_model.perplexity(X_count)
        if ppx < best_ppx :
            best_ppx = ppx
            best_topic_num = num_topics
    print(f"토픽 수 : {best_topic_num}, 해당 Perplexity : {best_ppx}")
        

In [50]:
# perplexity(df_bosa_test1['news_title'])

- 토픽 수가 적을 수록 perplexity가 적다(=좋다)고 나옴...

### 군집 만들기

In [51]:
components = LatentDirichletAllocation(n_components=3, n_jobs=-1)
components.fit(features)

LatentDirichletAllocation(n_components=3, n_jobs=-1)

In [52]:
dictionary_list = tfidfvectorizer.get_feature_names_out(features)
dictionary_list

array(['가격', '가능', '가능성', '가다', '가동', '가속', '가약', '가이드라인', '가장', '가정',
       '가제', '가족', '가치', '가톨릭대', '간담', '간질', '갈다', '감사패', '감소', '감염병',
       '강남', '강소', '강연', '강원', '강좌', '강직', '강화', '갖추다', '개강', '개년', '개다',
       '개발', '개방', '개선', '개설', '개성', '개시', '개원', '개인', '개정', '개최', '개편',
       '개혁', '거래', '거점', '건강', '건양대', '걷기', '걸다', '걸리다', '걸음', '검사', '검토',
       '게놈', '게재', '결과', '결산', '결정', '결핍', '경감', '경북', '경영', '경쟁', '경쟁력',
       '경제', '경제성', '경평', '경희대', '계속', '계약', '계획', '고가', '고대', '고려', '고민',
       '고셔병', '고속', '고통', '고혈압', '골수', '공감', '공개', '공급', '공단', '공동', '공략',
       '공모', '공식', '공유', '공익', '공학사', '공헌', '과제', '과학', '관계', '관련', '관심',
       '관절염', '광동', '교수', '교육', '교정', '구매', '구성', '구지원', '구진', '구축', '국가',
       '국감', '국립', '국민', '국산', '국제', '국회', '국훈', '권고', '권역별', '규명', '규모',
       '규제', '그룹', '극복', '근무', '근육', '글로벌', '글로벌화', '글리벡', '금지', '급등',
       '급성', '급증', '기관', '기금', '기념', '기능', '기대', '기독', '기린', '기반', '기부',
       '기술', '기업', '기전', '기존', '기준', '기증', '기

In [53]:
components.components_ # 토픽별로 단어의 확률 분포를 나타냄

array([[3.98764216, 0.34908106, 4.05228125, ..., 0.34555753, 0.36305411,
        0.33484671],
       [0.37389769, 3.86571105, 0.34396052, ..., 9.52544325, 7.93053043,
        0.33637751],
       [0.34419136, 4.07034686, 2.06016873, ..., 0.34262524, 3.74431846,
        3.56825789]])

In [54]:
topics_output = components.transform(features)
df_topics_score = pd.DataFrame(data=topics_output)

In [55]:
df_topics_score

,0,1,2
0,0.407988,0.111715,0.480297
1,0.540991,0.101541,0.357467
2,0.111708,0.327431,0.560861
3,0.138284,0.138287,0.723429
4,0.117111,0.190023,0.692866
...,...,...,...
2420,0.166741,0.179150,0.654109
2421,0.112548,0.122101,0.765350
2422,0.107484,0.579072,0.313444
2423,0.166835,0.166870,0.666295


In [56]:
import numpy as np
df_topics_score['topic_num'] = np.argmax(topics_output, axis=1)

In [57]:
df_topics_score['topic_num']

0       2
1       0
2       2
3       2
4       2
       ..
2420    2
2421    2
2422    1
2423    2
2424    1
Name: topic_num, Length: 2425, dtype: int64

In [58]:
df_bosa_test1

,news_title,news_url,news_when,news_topic,news_paper,news_contents
0,"한국쿄와기린, 세계 희귀질환의 날 행사 참여",http://www.bosa.co.kr/news/articleView.html?id...,2024.02.29,의료/법안,의학신문,[의학신문·일간보사=김영주 기자]‘세계 희귀질환의 날’을 맞아 한국쿄와기린은 그룹에...
1,"광동제약, ‘세계 희귀질환의 날’ 기념 환아 작품 전시회 개최",http://www.bosa.co.kr/news/articleView.html?id...,2024.02.29,신약/개발,의학신문,[의학신문·일간보사=김영주 기자]광동제약(대표 최성원)은 ‘세계 희귀질환의 날’을 ...
2,"순천향대 부천병원 신영림 교수, ‘질병관리청장 표창’ 수상",http://www.bosa.co.kr/news/articleView.html?id...,2024.02.29,심포지엄/행사,의학신문,[의학신문·일간보사=이상만 기자]순천향대 부천병원 소아청소년과 신영림 교수(사진)가...
3,"창립 70주년 한독, ‘THANKS CAMPAIGN’ 실시",http://www.bosa.co.kr/news/articleView.html?id...,2024.02.29,의료/법안,의학신문,"[의학신문·일간보사=김정일 기자] 한독(대표이사 김영진, 백진기)이 창립 70주년을..."
4,[세계 희귀질환의 날] 국가‧사회‧이웃‧의료진‧가족 함께 가야 한다,http://www.bosa.co.kr/news/articleView.html?id...,2024.02.29,신약/개발,의학신문,[의학신문·일간보사=오인규 기자] “이건 신이 실수를 하신 거다. 그러니 너무 자책...
...,...,...,...,...,...,...
2420,26일 `희귀질환 치료' 심포지엄,http://www.bosa.co.kr/news/articleView.html?id...,2001.05.22,의료/법안,의학신문,NaN
2421,`희귀^난치성질환자 연합회' 구성 추진,http://www.bosa.co.kr/news/articleView.html?id...,2001.05.10,신약/개발,의학신문,NaN
2422,책임운영기관 평가 “적정성 결여,http://www.bosa.co.kr/news/articleView.html?id...,2001.04.16,의료/법안,의학신문,NaN
2423,먹지도 굶지도 못하는 희귀질환 발견,http://www.bosa.co.kr/news/articleView.html?id...,2001.04.11,심포지엄/행사,의학신문,NaN


In [59]:
df_bosa_test1['topic_num'] = df_topics_score['topic_num']

In [60]:
# 그럼 이제 각각의 news가 어떤 topic으로 분류되는지는 알았는데
# 각 토픽이 어떤 단어를 가지고 있는지 알기 위해서는 상위 단어 추출이 필요함

# 상위 단어 추출

topics_list = []
for topic in components.components_ : # 토픽별로 단어의 확률 분포 나타낸 것
    df_datas = [topic, dictionary_list] # dictionary_list는 단어 나열
    df_topics = pd.DataFrame(data=df_datas).T
    df_topics = df_topics.sort_values(0, ascending=False)
    topics_text = ' '.join(df_topics[1].values[:5]) # 상위 4개를 시리즈 형식으로 출력하기
    print(topics_text) 
    topics_list.append(topics_text)

topics_list_table = [topics_list, ['Topic1', 'Topic2', 'Topic3']]
topics_list_table

신약 개발 제약 바이오 승인
의약품 확대 제휴 센터 연구
의료 개최 지원 평가 심포지엄


[['신약 개발 제약 바이오 승인', '의약품 확대 제휴 센터 연구', '의료 개최 지원 평가 심포지엄'],
 ['Topic1', 'Topic2', 'Topic3']]

In [61]:
topics_list_table = pd.DataFrame(topics_list_table)
topics_list_table

,0,1,2
0,신약 개발 제약 바이오 승인,의약품 확대 제휴 센터 연구,의료 개최 지원 평가 심포지엄
1,Topic1,Topic2,Topic3


In [62]:
pivot_table_title = pd.pivot_table(data=df_bosa_test1,
                                   values='news_title'
                                   ,index='topic_num'
                                   , aggfunc='count')
pivot_table_title

,news_title
topic_num,
0,915
1,763
2,747


### 시각화

In [63]:
vis = pyLDAvis.lda_model.prepare(components, features, tfidfvectorizer)
# 토픽모델, 교육이 끝난 값(행렬형태), 교육모델

In [64]:
pyLDAvis.enable_notebook()
pyLDAvis.display(vis) # PCA - 차원축소 

In [65]:
df_bosa_test1['topic_model'] = df_topics_score['topic_num']
df_bosa_test1[45:51]

,news_title,news_url,news_when,news_topic,news_paper,news_contents,topic_num,topic_model
45,제1기 권역별 '희귀질환 전문기관' 지정,http://www.bosa.co.kr/news/articleView.html?id...,2024.01.23,심포지엄/행사,의학신문,"[의학신문·일간보사=김상일 기자]""한국로슈 신경과학사업부는 신경과 영역에서 새로운 ...",1,1
46,"삼성바이오에피스, ‘에피스클리’ 국내 품목 허가",http://www.bosa.co.kr/news/articleView.html?id...,2024.01.22,의료/법안,의학신문,[의학신문·일간보사=이승덕 기자]질병청이 올해부터 의료비 지원대상을 1272개(이전...,0,0
47,"""환자를 위해 없던 길도 만들어 나간다""",http://www.bosa.co.kr/news/articleView.html?id...,2024.01.22,신약/개발,의학신문,"[의학신문·일간보사=김정일 기자] “희귀질환, 당뇨, 고혈압 등 전 비즈니스 영역을...",1,1
48,"GC녹십자, 산필리포증후군 치료제 유럽 희귀의약품 지정",http://www.bosa.co.kr/news/articleView.html?id...,2024.01.19,심포지엄/행사,의학신문,[의학신문·일간보사=김영주 기자]제약기업이 살 길은 역시 신약개발임이 여실히 드러났...,0,0
49,‘급성골수성백혈병’ 새로운 치료 옵션 등장 치료 환경 개선 움직임,http://www.bosa.co.kr/news/articleView.html?id...,2024.01.15,신약/개발,의학신문,[의학신문·일간보사=김정일 기자] 닥터노아바이오텍(대표 이지현)이 미국 뉴로덱스와 ...,0,0
50,[2024 다국적제약 우리 부서가 뛴다]사노피 희귀질환 사업부 마케팅팀,http://www.bosa.co.kr/news/articleView.html?id...,2024.01.15,신약/개발,의학신문,[의학신문·일간보사=유은제 기자]“국내 허가 되지 않은 약이었지만 저는 엄마이고 아...,2,2


In [66]:
type(df_bosa_test1['topic_model'][0])

numpy.int64

In [67]:
df_bosa_test1['topic_model'] = df_bosa_test1['topic_model'].apply(int)

In [68]:
# for idx, topic in enumerate(df_bosa_test1['topic_model']) :
#     if topic == 0 :
#         df_bosa_test1.loc[idx, 'topic_model'] = topics_list_table.loc[0][0]
#     elif topic == 1 :
#         df_bosa_test1.loc[idx, 'topic_model'] = topics_list_table.loc[0][1]
#     elif topic == 2 :
#         df_bosa_test1.loc[idx, 'topic_model'] = topics_list_table.loc[0][2]
# df_bosa_test1.head()

In [69]:
for idx, topic in enumerate(df_bosa_test1['topic_model']) :
    if topic == 0 :
        df_bosa_test1.loc[idx, 'topic_model'] = '심포지엄/행사'
    elif topic == 1 :
        df_bosa_test1.loc[idx, 'topic_model'] = '신약/개발'
    elif topic == 2 :
        df_bosa_test1.loc[idx, 'topic_model'] = '의료/법안'
df_bosa_test1.head()

/tmp/ipykernel_43941/354877451.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '의료/법안' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_bosa_test1.loc[idx, 'topic_model'] = '의료/법안'


,news_title,news_url,news_when,news_topic,news_paper,news_contents,topic_num,topic_model
0,"한국쿄와기린, 세계 희귀질환의 날 행사 참여",http://www.bosa.co.kr/news/articleView.html?id...,2024.02.29,의료/법안,의학신문,[의학신문·일간보사=김영주 기자]‘세계 희귀질환의 날’을 맞아 한국쿄와기린은 그룹에...,2,의료/법안
1,"광동제약, ‘세계 희귀질환의 날’ 기념 환아 작품 전시회 개최",http://www.bosa.co.kr/news/articleView.html?id...,2024.02.29,신약/개발,의학신문,[의학신문·일간보사=김영주 기자]광동제약(대표 최성원)은 ‘세계 희귀질환의 날’을 ...,0,심포지엄/행사
2,"순천향대 부천병원 신영림 교수, ‘질병관리청장 표창’ 수상",http://www.bosa.co.kr/news/articleView.html?id...,2024.02.29,심포지엄/행사,의학신문,[의학신문·일간보사=이상만 기자]순천향대 부천병원 소아청소년과 신영림 교수(사진)가...,2,의료/법안
3,"창립 70주년 한독, ‘THANKS CAMPAIGN’ 실시",http://www.bosa.co.kr/news/articleView.html?id...,2024.02.29,의료/법안,의학신문,"[의학신문·일간보사=김정일 기자] 한독(대표이사 김영진, 백진기)이 창립 70주년을...",2,의료/법안
4,[세계 희귀질환의 날] 국가‧사회‧이웃‧의료진‧가족 함께 가야 한다,http://www.bosa.co.kr/news/articleView.html?id...,2024.02.29,신약/개발,의학신문,[의학신문·일간보사=오인규 기자] “이건 신이 실수를 하신 거다. 그러니 너무 자책...,2,의료/법안


## 2. 머신러닝 학습

### 학습

In [70]:
features = features
target = df_bosa_test1['topic_model']

In [71]:
target.unique()

array(['의료/법안', '심포지엄/행사', '신약/개발'], dtype=object)

In [72]:
len(target)

2425

In [73]:
features.shape, target.shape

((2425, 997), (2425,))

In [74]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_jobs=-1)

In [75]:
from sklearn.model_selection import train_test_split
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.3, random_state=55)
features_train.shape, features_test.shape, target_train.shape, target_test.shape

((1697, 997), (728, 997), (1697,), (728,))

In [76]:
model.fit(features_train, target_train)

RandomForestClassifier(n_jobs=-1)

In [77]:
model.predict(features_test)

array(['심포지엄/행사', '신약/개발', '신약/개발', '심포지엄/행사', '의료/법안', '심포지엄/행사',
       '심포지엄/행사', '심포지엄/행사', '신약/개발', '심포지엄/행사', '심포지엄/행사', '심포지엄/행사',
       '심포지엄/행사', '신약/개발', '심포지엄/행사', '의료/법안', '심포지엄/행사', '신약/개발',
       '의료/법안', '의료/법안', '신약/개발', '신약/개발', '신약/개발', '의료/법안', '의료/법안',
       '의료/법안', '의료/법안', '심포지엄/행사', '의료/법안', '신약/개발', '심포지엄/행사', '의료/법안',
       '의료/법안', '심포지엄/행사', '심포지엄/행사', '의료/법안', '의료/법안', '심포지엄/행사',
       '신약/개발', '의료/법안', '신약/개발', '심포지엄/행사', '심포지엄/행사', '심포지엄/행사',
       '심포지엄/행사', '심포지엄/행사', '심포지엄/행사', '심포지엄/행사', '심포지엄/행사', '심포지엄/행사',
       '신약/개발', '의료/법안', '심포지엄/행사', '신약/개발', '심포지엄/행사', '심포지엄/행사',
       '신약/개발', '신약/개발', '심포지엄/행사', '심포지엄/행사', '심포지엄/행사', '신약/개발',
       '심포지엄/행사', '심포지엄/행사', '신약/개발', '심포지엄/행사', '신약/개발', '의료/법안',
       '신약/개발', '심포지엄/행사', '신약/개발', '심포지엄/행사', '신약/개발', '의료/법안', '신약/개발',
       '신약/개발', '심포지엄/행사', '신약/개발', '신약/개발', '신약/개발', '심포지엄/행사', '의료/법안',
       '심포지엄/행사', '신약/개발', '심포지엄/행사', '신약/개발', '심포지엄/행사', '의료/법안',
       '의료/법안', '신약/개발', '의료

In [78]:
from sklearn.metrics import classification_report

In [79]:
target_predict = model.predict(features_test)
print(classification_report(target_test, target_predict))

              precision    recall  f1-score   support

       신약/개발       0.75      0.69      0.72       234
     심포지엄/행사       0.64      0.85      0.73       267
       의료/법안       0.80      0.56      0.66       227

    accuracy                           0.71       728
   macro avg       0.73      0.70      0.70       728
weighted avg       0.73      0.71      0.71       728



### test

In [80]:
def testing(feature) : # 이때 feature는 리스트 형태
    token_feature = tfidfvectorizer.transform(feature)
    return model.predict(token_feature)

In [103]:
with open('news_recommend.pkl', 'wb') as file:
    pickle.dump(testing, file)

In [104]:
testing(["사지에 힘이 빠지고 저리다면, ‘길랑-바레 증후군’ 의심"])

array(['의료/법안'], dtype=object)

- f1-scroe : 0.73으로 준수한 성능을 보임

### Save

In [82]:
import pickle

In [83]:
# data = {'news_recommend_model' : model, 'news_recommend_vectorizer' : tfidfvectorizer}

In [117]:
with open('news_recommend_model.pkl', 'wb') as file:
    pickle.dump(model, file)

In [118]:
with open('news_recommend_vectorizer.pkl', 'wb') as file:
    pickle.dump(tfidfvectorizer, file)

In [119]:
with open('news_recommend_model.pkl', 'rb') as file:
    model_test = pickle.load(file)

In [120]:
with open('news_recommend_vectorizer.pkl', 'rb') as file:
    tfidf_test = pickle.load(file)

In [121]:
test_word = '사지에 힘이 빠지고 저리다면, ‘길랑-바레 증후군’ 의심'

In [122]:
tfidf_test.transform([test_word])

<1x997 sparse matrix of type '<class 'numpy.float64'>'
	with 2 stored elements in Compressed Sparse Row format>

In [123]:
model_test.predict(tfidf_test.transform([test_word]))

array(['의료/법안'], dtype=object)

## 3. 데이터 변경하기

In [91]:
df_bosa_test1.head()

,news_title,news_url,news_when,news_topic,news_paper,news_contents,topic_num,topic_model
0,"한국쿄와기린, 세계 희귀질환의 날 행사 참여",http://www.bosa.co.kr/news/articleView.html?id...,2024.02.29,의료/법안,의학신문,[의학신문·일간보사=김영주 기자]‘세계 희귀질환의 날’을 맞아 한국쿄와기린은 그룹에...,2,의료/법안
1,"광동제약, ‘세계 희귀질환의 날’ 기념 환아 작품 전시회 개최",http://www.bosa.co.kr/news/articleView.html?id...,2024.02.29,신약/개발,의학신문,[의학신문·일간보사=김영주 기자]광동제약(대표 최성원)은 ‘세계 희귀질환의 날’을 ...,0,심포지엄/행사
2,"순천향대 부천병원 신영림 교수, ‘질병관리청장 표창’ 수상",http://www.bosa.co.kr/news/articleView.html?id...,2024.02.29,심포지엄/행사,의학신문,[의학신문·일간보사=이상만 기자]순천향대 부천병원 소아청소년과 신영림 교수(사진)가...,2,의료/법안
3,"창립 70주년 한독, ‘THANKS CAMPAIGN’ 실시",http://www.bosa.co.kr/news/articleView.html?id...,2024.02.29,의료/법안,의학신문,"[의학신문·일간보사=김정일 기자] 한독(대표이사 김영진, 백진기)이 창립 70주년을...",2,의료/법안
4,[세계 희귀질환의 날] 국가‧사회‧이웃‧의료진‧가족 함께 가야 한다,http://www.bosa.co.kr/news/articleView.html?id...,2024.02.29,신약/개발,의학신문,[의학신문·일간보사=오인규 기자] “이건 신이 실수를 하신 거다. 그러니 너무 자책...,2,의료/법안


In [92]:
df_bosa_test1.drop(columns=['news_topic', 'topic_num'], inplace=True)

In [93]:
df_bosa_test1.head()

,news_title,news_url,news_when,news_paper,news_contents,topic_model
0,"한국쿄와기린, 세계 희귀질환의 날 행사 참여",http://www.bosa.co.kr/news/articleView.html?id...,2024.02.29,의학신문,[의학신문·일간보사=김영주 기자]‘세계 희귀질환의 날’을 맞아 한국쿄와기린은 그룹에...,의료/법안
1,"광동제약, ‘세계 희귀질환의 날’ 기념 환아 작품 전시회 개최",http://www.bosa.co.kr/news/articleView.html?id...,2024.02.29,의학신문,[의학신문·일간보사=김영주 기자]광동제약(대표 최성원)은 ‘세계 희귀질환의 날’을 ...,심포지엄/행사
2,"순천향대 부천병원 신영림 교수, ‘질병관리청장 표창’ 수상",http://www.bosa.co.kr/news/articleView.html?id...,2024.02.29,의학신문,[의학신문·일간보사=이상만 기자]순천향대 부천병원 소아청소년과 신영림 교수(사진)가...,의료/법안
3,"창립 70주년 한독, ‘THANKS CAMPAIGN’ 실시",http://www.bosa.co.kr/news/articleView.html?id...,2024.02.29,의학신문,"[의학신문·일간보사=김정일 기자] 한독(대표이사 김영진, 백진기)이 창립 70주년을...",의료/법안
4,[세계 희귀질환의 날] 국가‧사회‧이웃‧의료진‧가족 함께 가야 한다,http://www.bosa.co.kr/news/articleView.html?id...,2024.02.29,의학신문,[의학신문·일간보사=오인규 기자] “이건 신이 실수를 하신 거다. 그러니 너무 자책...,의료/법안


In [94]:
from datetime import datetime

In [95]:
df_bosa_test1['news_when'][1][:-6]

'2024'

In [96]:
def change_datetime(date):
    date = date[:-6]
    if len(date) == 5 :
        date = '2024.' + date
    else :
        pass
    return date

In [97]:
df_bosa_test1['news_when'] = df_bosa_test1['news_when'].apply(change_datetime)

In [98]:
df_bosa_test1['news_paper'] = '의학신문'

In [99]:
df_bosa_test1.head()

,news_title,news_url,news_when,news_paper,news_contents,topic_model
0,"한국쿄와기린, 세계 희귀질환의 날 행사 참여",http://www.bosa.co.kr/news/articleView.html?id...,2024,의학신문,[의학신문·일간보사=김영주 기자]‘세계 희귀질환의 날’을 맞아 한국쿄와기린은 그룹에...,의료/법안
1,"광동제약, ‘세계 희귀질환의 날’ 기념 환아 작품 전시회 개최",http://www.bosa.co.kr/news/articleView.html?id...,2024,의학신문,[의학신문·일간보사=김영주 기자]광동제약(대표 최성원)은 ‘세계 희귀질환의 날’을 ...,심포지엄/행사
2,"순천향대 부천병원 신영림 교수, ‘질병관리청장 표창’ 수상",http://www.bosa.co.kr/news/articleView.html?id...,2024,의학신문,[의학신문·일간보사=이상만 기자]순천향대 부천병원 소아청소년과 신영림 교수(사진)가...,의료/법안
3,"창립 70주년 한독, ‘THANKS CAMPAIGN’ 실시",http://www.bosa.co.kr/news/articleView.html?id...,2024,의학신문,"[의학신문·일간보사=김정일 기자] 한독(대표이사 김영진, 백진기)이 창립 70주년을...",의료/법안
4,[세계 희귀질환의 날] 국가‧사회‧이웃‧의료진‧가족 함께 가야 한다,http://www.bosa.co.kr/news/articleView.html?id...,2024,의학신문,[의학신문·일간보사=오인규 기자] “이건 신이 실수를 하신 거다. 그러니 너무 자책...,의료/법안


In [100]:
df_bosa_test1.to_csv('df_bosa.csv', index=False)

In [101]:
df_bosa_test1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2425 entries, 0 to 2424
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   news_title     2425 non-null   object
 1   news_url       2425 non-null   object
 2   news_when      2425 non-null   object
 3   news_paper     2425 non-null   object
 4   news_contents  2142 non-null   object
 5   topic_model    2425 non-null   object
dtypes: object(6)
memory usage: 113.8+ KB


In [102]:
df_bosa = pd.read_csv("./csv/bosa_news_all.csv")

FileNotFoundError: [Errno 2] No such file or directory: './csv/bosa_news_all.csv'

In [ ]:
df_bosa_test1['news_contents'] = df_bosa['news_contents']

In [ ]:
df_bosa_test1.head()

In [ ]:
df_bosa_test1.to_csv('df_bosa.csv', index=False)

In [ ]:
df_bosa_test1 = df_bosa_test1.rename(columns={'topic_model':'news_topic'})